## 0. 필요한 라이브러리 설치

In [ ]:
##############################################
# 0. 라이브러리 설치 (최초 1회 실행)
##############################################
%pip install --quiet python-dotenv azure-identity azure-search-documents openai

## 1. 환경 변수 설정

In [ ]:
import os
from dotenv import load_dotenv

##############################################
# 1. 환경 변수 로드 및 확인
##############################################
load_dotenv()

# Azure AI Search
AZURE_SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
AZURE_SEARCH_INDEX_NAME = os.getenv("AZURE_SEARCH_INDEX_NAME")

# Azure OpenAI
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_EMBEDDING_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01")

print(f"Search Endpoint: {AZURE_SEARCH_SERVICE_ENDPOINT}")
print(f"Search Index: {AZURE_SEARCH_INDEX_NAME}")
print(f"OpenAI Endpoint: {AZURE_OPENAI_ENDPOINT}")
print(f"Embedding Model: {AZURE_OPENAI_EMBEDDING_DEPLOYMENT}")

## 2. 클라이언트 초기화

In [ ]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.search.documents import SearchClient
from openai import AzureOpenAI
from IPython.display import display, HTML

##############################################
# 2-1. Azure 인증 및 클라이언트 생성
##############################################
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(
    credential,
    "https://cognitiveservices.azure.com/.default"
)

search_client = SearchClient(
    endpoint=AZURE_SEARCH_SERVICE_ENDPOINT,
    index_name=AZURE_SEARCH_INDEX_NAME,
    credential=credential
)

openai_client = AzureOpenAI(
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_ad_token_provider=token_provider
)

##############################################
# 2-2. 임베딩 생성 함수
##############################################
def get_embedding(text):
    response = openai_client.embeddings.create(
        input=text,
        model=AZURE_OPENAI_EMBEDDING_DEPLOYMENT
    )
    return response.data[0].embedding

print("클라이언트 초기화 완료")

## 3. 기본 키워드 검색 (가중치 없음)

In [ ]:
##############################################
# 3. 기본 키워드 검색 (필드 가중치 없음)
##############################################
search_query = "나이키"

results = search_client.search(
    search_text=search_query,
    top=5,
    query_type="full",
    search_fields=["name", "brand", "description"],
    select=["id", "name", "brand", "price", "description", "imageUrl"]
)

print(f"검색어: '{search_query}'")
print(f"\n{'='*60}")
print("[기본 키워드 검색 결과 - 가중치 없음]")
print(f"{'='*60}\n")

for idx, result in enumerate(results, 1):
    score = result.get('@search.score', 'N/A')
    print(f"{idx}. {result['name']} ({result['brand']})")
    print(f"   검색 점수: {score}")
    print(f"   가격: ₩{result['price']:,}")
    print(f"   설명: {result['description']}")
    display(HTML(f'<img src="{result["imageUrl"]}" width="200" />'))
    print()

## 4. 필드 가중치 적용 검색

필드별 가중치를 적용하여 특정 필드의 매칭을 더 중요하게 취급합니다.

**가중치:**
- `name^3`: 상품명 매칭 시 3배 가중
- `brand^2`: 브랜드 매칭 시 2배 가중
- `description^1`: 설명 매칭 시 기본 가중 (1배)

In [ ]:
##############################################
# 4. 필드 가중치 적용 검색
##############################################
search_query = "나이키"

results = search_client.search(
    search_text=search_query,
    top=5,
    query_type="full",
    search_fields=["name^3", "brand^2", "description^1"],  # 필드별 가중치 적용
    select=["id", "name", "brand", "price", "description", "imageUrl"]
)

print(f"검색어: '{search_query}'")
print(f"\n{'='*60}")
print("[필드 가중치 적용 검색 결과]")
print("가중치: name^3, brand^2, description^1")
print(f"{'='*60}\n")

for idx, result in enumerate(results, 1):
    score = result.get('@search.score', 'N/A')
    print(f"{idx}. {result['name']} ({result['brand']})")
    print(f"   검색 점수: {score}")
    print(f"   가격: ₩{result['price']:,}")
    print(f"   설명: {result['description']}")
    display(HTML(f'<img src="{result["imageUrl"]}" width="200" />'))
    print()

## 5. 가중치 효과 비교

동일한 검색어로 가중치 있음/없음을 비교하여 순위 변화를 확인합니다.

In [ ]:
##############################################
# 5. 가중치 효과 비교
##############################################
search_query = "아디다스 운동화"

# 5-1. 가중치 없음
results_no_weight = list(search_client.search(
    search_text=search_query,
    top=5,
    query_type="full",
    search_fields=["name", "brand", "description"],
    select=["id", "name", "brand", "price", "description"]
))

# 5-2. 가중치 적용
results_with_weight = list(search_client.search(
    search_text=search_query,
    top=5,
    query_type="full",
    search_fields=["name^3", "brand^2", "description^1"],
    select=["id", "name", "brand", "price", "description"]
))

print(f"검색어: '{search_query}'")
print(f"\n{'='*80}")
print("[가중치 효과 비교]")
print(f"{'='*80}\n")

print("📊 가중치 없음 (기본)")
print("-" * 80)
for idx, result in enumerate(results_no_weight, 1):
    score = result.get('@search.score', 'N/A')
    print(f"{idx}. [{score:>6}] {result['name']:<40} | {result['brand']:<15}")

print(f"\n📈 가중치 적용 (name^3, brand^2, description^1)")
print("-" * 80)
for idx, result in enumerate(results_with_weight, 1):
    score = result.get('@search.score', 'N/A')
    print(f"{idx}. [{score:>6}] {result['name']:<40} | {result['brand']:<15}")

print(f"\n{'='*80}")
print("💡 가중치를 적용하면 상품명과 브랜드에서 매칭된 항목이 상위에 노출됩니다.")

## 6. 하이브리드 검색 + 필드 가중치

키워드 검색에 필드 가중치를 적용하고, 벡터 검색을 결합하여 최상의 검색 결과를 얻습니다.

In [ ]:
from azure.search.documents.models import VectorizedQuery

##############################################
# 6. 하이브리드 검색 + 필드 가중치
##############################################
search_query = "여름에 입을 시원한 나이키 티셔츠"

# 검색어를 벡터로 변환
query_vector = get_embedding(search_query)

vector_query = VectorizedQuery(
    vector=query_vector,
    k_nearest_neighbors=5,
    fields="descriptionVector"
)

results = search_client.search(
    search_text=search_query,
    vector_queries=[vector_query],
    top=5,
    search_fields=["name^3", "brand^2", "description^1"],  # 필드 가중치 적용
    select=["id", "name", "brand", "price", "description", "imageUrl"]
)

print(f"검색어: '{search_query}'")
print(f"\n{'='*60}")
print("[하이브리드 검색 + 필드 가중치]")
print("키워드 검색 가중치: name^3, brand^2, description^1")
print("벡터 검색: descriptionVector (의미 기반 검색)")
print(f"{'='*60}\n")

for idx, result in enumerate(results, 1):
    score = result.get('@search.score', 'N/A')
    print(f"{idx}. {result['name']} ({result['brand']})")
    print(f"   통합 점수: {score}")
    print(f"   가격: ₩{result['price']:,}")
    print(f"   설명: {result['description']}")
    display(HTML(f'<img src="{result["imageUrl"]}" width="200" />'))
    print()

## 7. 실전 예제: 다양한 쿼리로 테스트

In [ ]:
##############################################
# 7. 다양한 검색 쿼리 테스트
##############################################

test_queries = [
    "아디다스",           # 브랜드 중심 검색
    "울트라부스트",        # 상품명 중심 검색
    "편안한 운동화",       # 설명 중심 검색
]

for query in test_queries:
    print(f"\n{'='*80}")
    print(f"검색어: '{query}'")
    print(f"{'='*80}")
    
    results = search_client.search(
        search_text=query,
        top=3,
        query_type="full",
        search_fields=["name^3", "brand^2", "description^1"],
        select=["name", "brand", "price", "description"]
    )
    
    for idx, result in enumerate(results, 1):
        score = result.get('@search.score', 'N/A')
        print(f"{idx}. [{score:>6}] {result['name']:<40} | {result['brand']:<15}")
        print(f"         {result['description'][:60]}...")
    
    print()

## 정리

### 필드 가중치 (Field Boosting)란?

- **목적**: 특정 필드에서 매칭된 결과에 더 높은 점수를 부여
- **문법**: `fieldName^boostValue`
  - 예: `name^3` = 상품명 필드에서 매칭 시 3배 점수

### 주요 사용 사례

1. **상품명 우선**: 사용자가 정확한 상품명을 검색할 때 상위 노출
2. **브랜드 중요도**: 브랜드 검색 시 관련 상품 우선 표시
3. **설명 보조**: 상세 설명은 보조적으로 활용

### 하이브리드 검색과의 조합

- **키워드 검색**: 필드 가중치로 정확한 매칭 우선
- **벡터 검색**: 의미적 유사성으로 관련 상품 발견
- **RRF (Reciprocal Rank Fusion)**: 두 검색 결과를 통합하여 최적의 순위 생성

### 다음 단계

이제 **07_image_to_product_info.ipynb**에서 이미지 분석을 통한 상품 정보 추출을 학습합니다.